In [67]:
# !pip install ipykernel
# !pip install --upgrade nbformat
# !pip install bokeh
# !pip install plotly

# **TRỰC QUAN HOÁ DỮ LIỆU**

<hr />

## **Import Thư viện**

In [68]:
# from bokeh.plotting.figure import figure
import plotly.express as px
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import matplotlib.pyplot as plt
import os
#Nếu đã có file populationForecast bằng cách chạy file "4 model.ipynb"
#Có thể trực quan bằng cách bỏ dấu # ở 3 dòng tiếp theo đây
# df1=pd.read_csv('./population.csv')
# df2=pd.read_csv('./populationForecast.csv')
# df=pd.DataFrame.append(df1,df2)

df=pd.read_csv('./population.csv')

#Các biểu đồ được lưu dưới dạng html ở folder "./3 visualization"
if not os.path.exists("./3 visualization"):
    os.makedirs('./3 visualization')

### **Dân số các nước được phân theo châu lục như thế nào?**

Vì số quốc gia lên tới hơn 270 quốc gia nên không thể biểu diễn trên cùng một biểu đồ được. Vì thế giải pháp là tạo ra một thanh cuộn theo năm, dữ liệu sẽ được thay đổi trực tiếp trên khung chính, dẫn tới dữ liệu hiển thị trực quan hơn, dễ nhân ra sự khác biệt hơn và đưa ra được nhiều nhận xét hơn.

In [69]:
fig = px.bar(df, x="Country", y="Population", animation_frame="Year", animation_group="Country", color="Continent", log_y=True,range_y=[10,10000000000],height=900)


fig.update_layout(xaxis=dict(rangeslider=dict(visible=True)))
fig.update_xaxes(tickangle=-45)
sliders=fig['layout']['sliders']
updatemenus=fig['layout']['updatemenus']

fig['layout']['updatemenus'][0]['pad']=dict(r= 20, t= 270)
fig['layout']['sliders'][0]['pad']=dict(r= 10, t= 250)
fig.update_layout(
    yaxis={"tickmode": "array",
          "tickvals": pd.to_numeric([f"{n:.1g}" for n in np.geomspace(1, df["Population"].max(), 25)])
           })
fig.update_layout(
    title={
        'text': "Biểu đồ thống kê dân số các nước trên thế giới",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.write_html(f"./3 visualization/1 {fig['layout']['title']['text']}.html")
fig.show() 

### **Tổng dân số trên thế giới phân theo châu lục như thế nào?**

Bên cạnh việc thể hiện tổng dân số thế giới theo châu lục, ta biểu thị sự thay đổi dân số theo năm trên cùng một biểu đồ miền.

In [70]:
continent_df=df.sort_values(by=['Continent'],ascending=True)[['Year','Population','Continent']].groupby(['Year','Continent'],group_keys=True).sum().reset_index()
fig = px.area(continent_df, x="Year", y="Population", color="Continent", line_group="Continent")
for i in range(len(fig['data'])):
  fig['data'][i]['line']['width']=0
fig.update_layout(
    title={
        'text': "Biểu đồ phân miền dân số thế giới theo châu lục",
        'y':0.9,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.write_html(f"./3 visualization/2 {fig['layout']['title']['text']}.html")
fig.show()

Qua biểu đồ trên ta thấy được dân số thế giới tăng đều tuyến tính qua các năm, trong đó **Châu Á (Asia)** là châu lục có dân số cao nhất, và **Châu Đại Dương (Oceania)** có dân số thấp nhất

### **Tỉ lệ dân số qua các năm**

Về tỉ lệ ta sẽ vẽ biểu đồ tròn để trực quan, có 2 biểu đồ: Châu lục và quốc gia. Trong đó, độ lớn của biểu đồ biểu thị cho tổng dân số.

In [71]:
fig_dict = {
    "data": [],
    "layout": {},
    "frames": []
}

world_pop={i : sum(df[df['Year']==i]['Population']) for i in df['Year'].unique()}
minyear=min(df['Year'].unique())
df_=df[df['Year']==minyear]
df_=df_.sort_values(by=['Population'],ascending=False).reset_index(drop=True)
df_.loc[df_['Population'] < df_.iloc[20]['Population'], 'Country'] = 'Other'

df_ = df_.groupby('Country')['Population'].sum().reset_index()

df_.index=df_.pop('Country')
df_=df_[['Population']].T


continent_df_=continent_df[continent_df['Year']==minyear]
continent_df_.index=continent_df_.pop('Continent')
continent_df_=continent_df_[['Population']].T

fig_dict["data"] = [
                        go.Pie(labels=continent_df_.columns,
                           title=f"Population by continent : {minyear}",
                           titleposition='bottom center',
                           values=continent_df_.iloc[0],
                           sort=True,
                           domain={'x': [0, 0.5], 'y': [0.0, 0.5+0.5*(world_pop[minyear]/max(world_pop.values()))]},
                           textinfo='percent+label',
                           textposition='inside',
                           hovertemplate= 'Continent=%{label}<br>Population=%{value}<extra></extra>'),
                        go.Pie(labels=df_.columns,
                           title=f"Population by country : {minyear}",
                           titleposition='bottom center',
                           values=df_.iloc[0],
                           sort=True,
                           domain={'x': [0.5, 1], 'y': [0, 0.5+0.5*(world_pop[minyear]/max(world_pop.values()))]},
                           textinfo='percent+label',
                           textposition='inside',
                           hovertemplate= 'Country=%{label}<br>Population=%{value}<extra></extra>'),]
frames_list = []
sliders_dict ={'steps':[]}
for index in df['Year'].unique() :
    df_=df[df['Year']==index]
    df_=df_.sort_values(by=['Population'],ascending=False).reset_index(drop=True)
    df_.loc[df_['Population'] < df_.iloc[20]['Population'], 'Country'] = 'Other'
    df_ = df_.groupby('Country')['Population'].sum().reset_index()
    df_.index=df_.pop('Country')
    df_=df_[['Population']].T
    continent_df_=continent_df[continent_df['Year']==index]
    continent_df_.index=continent_df_.pop('Continent')
    continent_df_=continent_df_[['Population']].T
    frames_list.append(
        go.Frame(data=[
                        go.Pie(labels=continent_df_.columns,
                           title="Population by continent : "+str(index),
                           titleposition='bottom center',
                           values=continent_df_.iloc[0],
                           sort=True,
                           domain={'x': [0.0, 0.5], 'y': [0.0, 0.5+0.5*(world_pop[index]/max(world_pop.values()))]},
                           textinfo='percent+label',
                           textposition='inside',
                           hovertemplate= 'Continent=%{label}<br>Population=%{value}<extra></extra>'),
                        go.Pie(labels=df_.columns,
                           title="Population by country : "+str(index),
                           titleposition='bottom center',
                           values=df_.iloc[0],
                           sort=True,
                           domain={'x': [0.5, 1], 'y': [0.0, 0.5+0.5*(world_pop[index]/max(world_pop.values()))]},
                           textinfo='percent+label',
                           textposition='inside',
                           hovertemplate= 'Country=%{label}<br>Population=%{value}<extra></extra>'),]
                           ,
                name= str(index))
    )
    slider_step = {'args': [
        [index],

       {'frame': {'duration': 0, 'redraw': True},
                        'mode': 'immediate', 'fromcurrent': True, 'transition':
                        {'duration': 0, 'easing': 'linear'}}
     ],
     'label': str(index),
     'method': 'animate'}
    sliders_dict['steps'].append(slider_step)

# create buttons for animations : Play, Pause
fig_dict["layout"]["sliders"] =sliders
fig_dict["layout"]["updatemenus"] =updatemenus
fig_dict['layout']['updatemenus'][0]['pad']=dict(r= 20, t= 0)
fig_dict['layout']['sliders'][0]['pad']=dict(r= 10, t= 0)
# add our list of frames to our dictionary
fig_dict["frames"] = frames_list
# create final figure
fig = go.Figure(fig_dict)
fig.update_layout(height=int(600))
fig.update_layout(
    title={
        'text': "Biểu đồ phần trăm thị phần dân số thế giới theo lục địa và đất nước",
        'y':0.9,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})


fig.write_html(f"./3 visualization/3 {fig['layout']['title']['text']}.html")
fig.show()




Ta thấy được sự áp đảo của Châu Á và 2 nước Trung Quốc và Ấn Độ.

### **Dân số và tỉ lệ dân thành thị thay đổi như thế nào?**

Một phân tích thú vị cho ta biết được xu hướng phát triển của 1 quốc gia là như thế nào. Theo dự đoán thì các quốc gia có xu hướng tăng tỉ lệ dân thành thị vì trong quá trình phát triển phải hiện đại hoá đất nước.

In [72]:
fig=px.scatter(df, x="Population", y="UrbanPop%", animation_frame="Year", animation_group="Country",
        color="Continent", hover_name="Country",
           log_x=True, range_x=[100,10000000000], range_y=[2,105])

fig.update_layout(
    title={
        'text': "Biểu đồ thể hiện mối liên hệ giữa dân số và tỉ lệ dân số ở thành thị",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.write_html(f"./3 visualization/4 {fig['layout']['title']['text']}.html")
fig.show()

Ta kéo trục thì sẽ dễ thấy được sự thay đổi của dồ thị đi về góc phải trên của đồ thị. Khi mà dân số tăng cao thì tỉ lệ dân thành thị cũng sẽ tăng theo. Hãy cùng quan sát 2 chấm ngoài cùng là Trung Quốc và Ấn Độ, ta có thể nhận định được rằng Trung Quốc phát triển nhanh hơn Ấn Độ rất nhiều.

### **Mối liên hệ giữa dân số và độ tuổi trung bình**

Chất lượng cuộc sống cũng như phúc lợi xã hội càng ngày các phát triển, có thể độ tuổi trung bình cũng như dân số cũng sẽ có chiều hướng tăng tuyến tính.

In [73]:
fig=px.scatter(df, x="Population", y="MedianAge", animation_frame="Year", animation_group="Country",
        color="Continent", hover_name="Country",
           log_x=True, range_x=[10000,10000000000], range_y=[10,50])
fig.update_layout(
    title={
        'text': "Biểu đồ thể hiện mối liên hệ giữa dân số và độ tuổi trung bình",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.write_html(f"./3 visualization/5 {fig['layout']['title']['text']}.html")
fig.show()

Có lẽ nó chỉ đúng với các nước phát triển. Nếu quan sát kỹ sự thay đổi của Châu Phi (Africa) ta chỉ thấy những chấm tròn có xu hướng đi ngang.

Vào các năm 1950 thì dân số trên thế giới có độ tuổi rất thấp, cao nhất là Châu Âu, và Châu Âu cũng là Châu lục chỉ có chiều hướng đi lên, có nghĩa là việc kiểm soát dân số khá tốt và sự phát triển quốc gia rất ấn tượng.

### **Có nước nào có tỉ lệ dân thành thị giảm gần đây không? Giải thích vì sao.**

Trong cuộc đua phát triển quốc gia, công nghiệp hoá, hiện đại hoá đất nước. Có lẽ như việc gia tăng tỉ lệ dân thành thị là rất khó tránh khỏi. Đây là điều mà bất cứ quốc gia nào cũng quan tâm khi mà tỉ lệ dân thành thị tăng sẽ kéo theo nhiều vấn đề nhức nhối không chỉ cho thành phố mà còn cả những vùng ngoại ô. Giảm tỉ lệ dân thành thị là điều đáng để quan tâm lúc này.

Ta lấy 3 năm 2018, 2019, 2020 để kiểm tra sự giảm tỉ lệ này.

In [74]:
df_city=df[df['Year'].isin([2018,2019,2020])]
city_urban_desc=[]

for i in df_city['Country'].unique():
    df_=df_city[df_city['Country']==i]
    df_=df_.sort_values(by=['UrbanPop%','Year'],ascending=True)
    if (False not in list(df_['Year']==[2020,2019,2018])) :
        city_urban_desc.append(i)

fig = px.line(df[df['Country'].isin(city_urban_desc)], x="Year", y="UrbanPop%", color='Country')
fig.update_layout(
    title={
        'text': "Biểu đồ thể hiện tỉ lệ dân thành thị của một số nước có xu hướng giảm",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.write_html(f"./3 visualization/6 {fig['layout']['title']['text']}.html")
fig.show()


**Vỏn vẹn chỉ có 13 quốc gia.**

Tất nhiên ta có thể chỉ học hỏi các nước phát triển hơn đất nước ta để giải quyết vấn đề tồn đọng, trong đó có Germany và Singapore. Các nước đang phát triển thường có sự di dân sang các nước phát triển hơn nên tỉ lệ dân thành thị giảm có thể do điều này.

### **Những quốc gia có xu hướng độ tuổi trung bình cao là quốc gia có kinh tế như thế nào (thuộc châu lục nào)?.**


In [75]:
df_high_age=df[df['Year']==2020]
df_high_age=df_high_age[df_high_age['MedianAge']>=40]
df_high_age=df_high_age.groupby(['Continent']).size().reset_index(name='High Median Age')
df_age=df[df['Year']==2020]
df_age=df_age.groupby(['Continent']).size().reset_index(name='counts')
df_age=df_age.merge(df_high_age,on='Continent', how='left').fillna(0)
df_age['Low Median Age']=df_age['counts']-df_age['High Median Age']
fig = px.bar(df_age, x="Continent", y=["Low Median Age", "High Median Age"])
fig.update_layout(
    title={
        'text': "Biểu đồ thể hiện sự liên quan giữa độ tuổi trung bình cao với châu lục",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.write_html(f"./3 visualization/7 {fig['layout']['title']['text']}.html")
fig.show()


Qua biểu đồ ta thấy số nước châu Âu có dân số già nhiều hơn số nước châu Âu có dân số trẻ. Ngược lại Châu Phi lại có dân số trẻ gần như là toàn bộ lục địa này. Điều này phân nào nói lên mối liên hệ tương quan giữa độ tuổi trung bình với trình độ phát triển của châu lục. Khi mà các nước phát triển có dân số già hơn các nước nghèo đói hoặc đang phát triển.

 **Điều này có thể giải thích bởi:**
 
- Các nước có dân số già với trình độ cao về kĩ thuật y tế dẫn đến tuổi thọ trung bình của những nước này tăng.
- Các nước nghèo không thể đảm bảo về y tế, việc dân số quá trẻ mà trình độ phát triển không đảm bảo lại khiến sự nghèo đói càng trầm trọng hơn.

=> Qua đó dân số phần nào thể hiện được bộ mặt kinh tế của một nước, một châu lục.

**Mối tương quan giữa độ tuổi trung bình với tỉ lệ tăng dân số.**


In [76]:
import math
df_=df[::]
#df_['Population']=[math.log(i,10) for i in df_.pop('Population')]
fig=px.scatter(df_, x="Yearly%Change", y="MedianAge", animation_frame="Year", animation_group="Country",size='Population',size_max=100,
        color="Continent", hover_name="Country", range_y=[10,55],height=800,range_x=[])
fig.update_layout(
    title={
        'text': "Biểu đồ thể hiện mối tương quan giữa độ tuổi trung bình với tỉ lệ tăng dân số",
        'y':0.95,
        'x':0.45,
        'xanchor': 'center',
        'yanchor': 'top'})

fig.write_html(f"./3 visualization/8 {fig['layout']['title']['text']}.html")
fig.show()

Ta dễ dàng nhận ra, khi tuổi trung bình của các cuốc gia càng tăng thì tỉ lệ gia tăng dân số lại càng giảm. Vào năm 2020, nhận xét này càng được thể hiện rõ khi các nước gần như tạo thành 1 đường thẳng hồi quy tuyến tính. Điều này có thể do độ tuổi trung bình trẻ dẫn tới tỉ lệ sinh nở nhanh hơn kéo theo sự phát triển nhanh của dân số.